In [72]:
import main_module as m
import zoho_filter_module as fil
import pandas as pd
import re
import numpy as np
import datetime

%load_ext autoreload
%autoreload 2

cols = ['Record Id',	
'Created Time',
'Created Date',	
'Booker First Name',	
'Booker Last Name',	
'Email',	
'Phone',	
'Mobile',	
'Birthdate',	
'Year of Birth',
'Street',	
'Street2',	
'City',	
'State',	
'Country',	
'Nationality',
'Booking Status', 
'BookRef',
'Departure Date',	
'Arrival Date',	
'Card Type',	
'Marital Status',	
'Lead Brand',	
'Lead Sub-Brand',	
'Lead Source',	
'Lead Locations',	
'Lead Source Description',	
'Lead Regions',
'Lead Status',
'Email Opt Out',	
'Opt In',
'GDPR Compliant',
'Website',
'Comments',
'Do you live in Bali?']


def reorder_date_from_yyyy(date_str):
    if date_str.startswith("20"):  # deteksi jika dimulai dengan tahun
        try:
            date_part, time_part = date_str.split(" ")
            yyyy, mm, dd = date_part.split("-")
            return f"{mm}-{dd}-{yyyy} {time_part}"
        except Exception:
            pass
    return date_str  # kembalikan asli jika bukan format yang dimaksud




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
path = r'C:\Users\fajar.fatoni\Documents\Python\Data\KLOM,KSH,KSM Checked Out - NEW.xlsx'
# zoho_df = pd.read_csv(path, dtype='string')
zoho_df = pd.read_excel(path, dtype='string', sheet_name='KLOM')

zoho_df.reset_index(drop=True, inplace=True)
count_filter_awal = zoho_df.shape[0]

# mengisi kolom <NA> dengan value string 'empty' supaya mudah mengecek data member dan booking dengan isin
zoho_df.fillna('empty', inplace=True)

zoho_df['Departure_Date'] = zoho_df['Departure_Date'].apply(reorder_date_from_yyyy)

zoho_df

,Resort Name,Departure_Date,Booking Id,Booker Title,Booker First Name,Booker Last Name,Email,TelNo,Concat_Tel_Mob,Media_Group,Source,MarketingOptIn,Market_Segment,Arrival Status,Departure Date
0,Karma Lake of Menteith,31-12-2024 00:00:00,KLM007575,Mrs,Hazel,Anderson,hazel.anderson@rocketmail.com,07870229780,7870229780,RETAIL,empty,Yes,DIGITAL,Arrival Completed,empty
1,Karma Lake of Menteith,30-12-2024 00:00:00,KLM007452,Mr,David,Collin,dcollin07837@gmail.com,07837855439,7837855439,DIGITAL,empty,Yes,DIGITAL,Arrival Completed,empty
2,Karma Lake of Menteith,30-12-2024 00:00:00,KLM006985,Mr,Paul,McKenna,carole.ann95@gmail.com,empty,7990750306,RETAIL,empty,Yes,DIR,Arrival Completed,empty
3,Karma Lake of Menteith,26-12-2024 00:00:00,KLM007507,Mr,Alan,Riddell,alanriddell11@gmail.com,07901544137,7901544137,RETAIL,empty,Yes,DIGITAL,Arrival Completed,empty
4,Karma Lake of Menteith,26-12-2024 00:00:00,KLM007062,Mrs,Kathryn,Foley,dkfoley85@gmail.com,44 7910 073758,447910073758,RETAIL,empty,Yes,DIR,Arrival Completed,empty
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
636,Karma Lake of Menteith,2025-06-01 00:00:00,empty,empty,Alicia,Gwen DSouza,aliciagwendsouza@gmail.com,empty,empty,empty,empty,Yes,WEDDING,Arrival Completed,empty
637,Karma Lake of Menteith,2025-06-01 00:00:00,empty,empty,Anna,Katharina Eichhorn,anna.eichhorn@freenet.de,empty,empty,empty,empty,Yes,WEDDING,Arrival Completed,empty
638,Karma Lake of Menteith,2025-06-01 00:00:00,empty,empty,Tobias,Züchner,Tobi.zuechner@icloud.com,empty,empty,empty,empty,Yes,WEDDING,Arrival Completed,empty
639,Karma Lake of Menteith,2025-06-01 00:00:00,empty,empty,Daisy,Mashburn,dafay13@gmail.com,empty,empty,empty,empty,Yes,WEDDING,Arrival Completed,empty


In [54]:
path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04eXpvS1NXTWF2RVE\_BOOKING\Booking By Year\FlyBuy Booking Data (BGAU & BGID & KE).xlsx'
df_booking = pd.read_excel(path, dtype='string', sheet_name='BOOKING_BGAU_BGID_KE')

path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Phone Members.xlsx'
df_phone = pd.read_excel(path, dtype='string', sheet_name='Sheet1')

path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Email Members.xlsx'
df_email = pd.read_excel(path, dtype='string', sheet_name='Sheet1')


KeyboardInterrupt: 

# ====================== Awal Filter Umum ====================== 

In [ ]:
# data phone, mobile dam email member yang ada di master data (wash member)

m.clean_number(zoho_df, 'TelNo')
m.clean_number(zoho_df, 'Concat_Tel_Mob')
m.replace_not_valid_to_empty(zoho_df, 'TelNo')
m.replace_not_valid_to_empty(zoho_df, 'Concat_Tel_Mob')
zoho_df['Email'] = m.lowercase(zoho_df, 'Email')
m.clean_space(zoho_df, 'Email')
m.clean_not_valid_email_format(zoho_df, 'Email')

m.clean_number(df_booking, 'Phone1')
m.clean_number(df_booking, 'Mobile')
df_booking['Email'] = m.lowercase(df_booking, 'Email')
m.clean_space(df_booking, 'Email')
m.clean_not_valid_email_format(df_booking, 'Email')

m.clean_number(df_phone, 'Mobile')

df_email['Email'] = m.lowercase(df_email, 'Email')
m.clean_space(df_email, 'Email')
m.clean_not_valid_email_format(df_email, 'Email')

c:\Users\fajar.fatoni\Documents\Python\Task Offline\zoho_data_app\main_module.py:104: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[df_columns].replace(' ','', regex=True, inplace=True)
c:\Users\fajar.fatoni\Documents\Python\Task Offline\zoho_data_app\main_module.py:105: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are set

'Email'

In [ ]:
count_deleted_wrong_format = m.delete_email_wrong_format(zoho_df, 'Email')
count_after_deleted_wrong_format = zoho_df.shape[0]

# ====================== Akhir Filter Umum ====================== 

In [ ]:
# hitung jumlah duplicate
count_duplicate_general = zoho_df.loc[zoho_df.duplicated(subset=['TelNo','Concat_Tel_Mob','Email'])].shape[0]

zoho_df.loc[zoho_df.duplicated(subset=['TelNo','Concat_Tel_Mob','Email'])]

,Resort Name,Departure_Date,Booking Id,Booker Title,Booker First Name,Booker Last Name,Email,TelNo,Concat_Tel_Mob,Media_Group,Source,MarketingOptIn,Market_Segment,Arrival Status,Departure Date
5,Karma Lake of Menteith,26 Dec 2024,KLM007508,Mr,Alan,Riddell,alanriddell11@gmail.com,07901544137,7901544137,RETAIL,empty,Yes,DIGITAL,Arrival Completed,December 2024
29,Karma Lake of Menteith,16 Oct 2024,KLM005163,empty,empty,empty,empty,empty,empty,GROUP,empty,Yes,DIR,Arrival Completed,October 2024
43,Karma Lake of Menteith,13 Oct 2024,KLM004471,Bride & Groo,Annie,Kennedy,anniekennedy02@gmail.com,empty,7956386778,GROUP,empty,Yes,DIR,Arrival Completed,October 2024
49,Karma Lake of Menteith,10 Dec 2024,KLM006652,Ms,Diana,Carmody,dianacarmody@gmail.com,07931161625,7931161625,GROUP,empty,Yes,DIR,Arrival Completed,empty
68,Karma Lake of Menteith,22 Sep 2024,KLM006699,Mrs,Kathryn,Foley,dkfoley85@gmail.com,447910073758,447910073758,RETAIL,empty,Yes,OTA,Arrival Completed,September 2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444,Karma Lake of Menteith,15 Apr 2025,KLM005833,Ms,Lynda,Weir,ljweir85@gmail.com,61452582905,61452582905,GROUP,empty,Yes,DIR,Arrival Completed,April 2025
447,Karma Lake of Menteith,31 Mar 2025,KLM001751,empty,empty,empty,empty,empty,empty,GROUP,empty,Yes,COMP,Arrival Completed,March 2025
457,Karma Lake of Menteith,15 Mar 2025,KLM008703,Ms,Carolyn,Morning,c.morning@argyll-international.com,7743353867,7743353867,RETAIL,empty,Yes,DIGITAL,Arrival Completed,March 2025
466,Karma Lake of Menteith,03 Apr 2025,KLM004147,Miss,Emilia,And Micheal,december.2512@yahoo.co.uk,447766689379,447766689379,GROUP,empty,Yes,COMP,Arrival Completed,empty


In [ ]:
zoho_df.drop_duplicates(subset=['TelNo','Concat_Tel_Mob','Email'], inplace=True)
zoho_df.reset_index(drop=True, inplace=True)

# hitung jumlah setelah value duplicate dihapus
count_after_dedup_duplicate = zoho_df.shape[0]

zoho_df

,Resort Name,Departure_Date,Booking Id,Booker Title,Booker First Name,Booker Last Name,Email,TelNo,Concat_Tel_Mob,Media_Group,Source,MarketingOptIn,Market_Segment,Arrival Status,Departure Date
0,Karma Lake of Menteith,31 Dec 2024,KLM007575,Mrs,Hazel,Anderson,hazel.anderson@rocketmail.com,07870229780,7870229780,RETAIL,empty,Yes,DIGITAL,Arrival Completed,December 2024
1,Karma Lake of Menteith,30 Dec 2024,KLM007452,Mr,David,Collin,dcollin07837@gmail.com,07837855439,7837855439,DIGITAL,empty,Yes,DIGITAL,Arrival Completed,December 2024
2,Karma Lake of Menteith,30 Dec 2024,KLM006985,Mr,Paul,McKenna,carole.ann95@gmail.com,empty,7990750306,RETAIL,empty,Yes,DIR,Arrival Completed,December 2024
3,Karma Lake of Menteith,26 Dec 2024,KLM007507,Mr,Alan,Riddell,alanriddell11@gmail.com,07901544137,7901544137,RETAIL,empty,Yes,DIGITAL,Arrival Completed,December 2024
4,Karma Lake of Menteith,26 Dec 2024,KLM007062,Mrs,Kathryn,Foley,dkfoley85@gmail.com,447910073758,447910073758,RETAIL,empty,Yes,DIR,Arrival Completed,December 2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
543,Karma Lake of Menteith,06 Jan 2025,empty,empty,Alicia,Gwen DSouza,aliciagwendsouza@gmail.com,empty,empty,empty,empty,Yes,WEDDING,Arrival Completed,empty
544,Karma Lake of Menteith,06 Jan 2025,empty,empty,Anna,Katharina Eichhorn,anna.eichhorn@freenet.de,empty,empty,empty,empty,Yes,WEDDING,Arrival Completed,empty
545,Karma Lake of Menteith,06 Jan 2025,empty,empty,Tobias,Züchner,tobi.zuechner@icloud.com,empty,empty,empty,empty,Yes,WEDDING,Arrival Completed,empty
546,Karma Lake of Menteith,06 Jan 2025,empty,empty,Daisy,Mashburn,dafay13@gmail.com,empty,empty,empty,empty,Yes,WEDDING,Arrival Completed,empty


In [ ]:
# Cek leads test
test_leads = zoho_df.loc[
    
        zoho_df['Email'].str.contains('^test@', flags=re.I, regex=True) |
        zoho_df['Email'].str.contains('\+test', flags=re.I, regex=True) |
    
        zoho_df['Booker First Name'].str.contains('^test$', flags=re.I, regex=True) |
        zoho_df['Booker Last Name'].str.contains('^test$', flags=re.I, regex=True) |

        zoho_df['Booker First Name'].str.contains('test 1', flags=re.I, regex=True) |
        zoho_df['Booker Last Name'].str.contains('test 2', flags=re.I, regex=True)|
        zoho_df['Booker First Name'].str.contains('test1', flags=re.I, regex=True) |
        zoho_df['Booker Last Name'].str.contains('test2', flags=re.I, regex=True)|

        zoho_df['Booker First Name'].str.contains(' test ', flags=re.I, regex=True) | 
        zoho_df['Booker Last Name'].str.contains(' test ', flags=re.I, regex=True) | 
        zoho_df['Booker Last Name'].str.contains(' test$', flags=re.I, regex=True)
    ]

count_leads_test = test_leads.shape[0]

# test_leads.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\test_leads_2.xlsx', index=False)
test_leads

,Resort Name,Departure_Date,Booking Id,Booker Title,Booker First Name,Booker Last Name,Email,TelNo,Concat_Tel_Mob,Media_Group,Source,MarketingOptIn,Market_Segment,Arrival Status,Departure Date
95,Karma Lake of Menteith,23 Aug 2024,KLM003194,Mr,John Baptiste,Puppi,test@karmalakeofmenteith.com,0033689937441,33689937441,RETAIL,empty,Yes,DIR,Arrival Completed,August 2024
511,Karma Lake of Menteith,06 Jan 2025,empty,empty,test,empty,test@gmail.coom,empty,empty,empty,empty,Yes,WEDDING,Arrival Completed,empty


In [ ]:
zoho_df.drop(test_leads.index, inplace=True)
zoho_df.reset_index(drop=True, inplace=True)

count_after_dedup_test = zoho_df.shape[0]

In [ ]:
# dnm_leads = zoho_df.loc[(zoho_df['Do Not Mail'] == 'true') | (zoho_df['Do Not Contact'] == 'true')]
# count_dnm = dnm_leads.shape[0]

In [ ]:
# zoho_df.drop(dnm_leads.index, inplace=True)
# zoho_df.reset_index(drop=True, inplace=True)

# count_after_dedup_dnm = zoho_df.shape[0]

In [ ]:
# Cek Email OTA
email_OTA = m.find_OTA(zoho_df, 'Email')
count_email_OTA = email_OTA.shape[0]
# email_OTA.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\email_OTA.xlsx', index=False)

email_OTA

,Resort Name,Departure_Date,Booking Id,Booker Title,Booker First Name,Booker Last Name,Email,TelNo,Concat_Tel_Mob,Media_Group,Source,MarketingOptIn,Market_Segment,Arrival Status,Departure Date
84,Karma Lake of Menteith,09 May 2024,KLM006180,empty,Hans,Marijnen,hmarij.583245@guest.booking.com,31627020026,31627020026,RETAIL,empty,Yes,OTA,Arrival Completed,empty
102,Karma Lake of Menteith,17 Aug 2024,KLM006297,empty,Craig,Baskin,cbaski.323902@guest.booking.com,447539204472,447539204472,RETAIL,empty,Yes,OTA,Arrival Completed,August 2024
116,Karma Lake of Menteith,24 Jul 2024,KLM006064,empty,Stuart,Maclennan,smacle.949751@guest.booking.com,447733528673,447733528673,RETAIL,empty,Yes,OTA,Arrival Completed,July 2024
130,Karma Lake of Menteith,19 Jul 2024,KLM005999,empty,Mario,Wagner,mwagne.635451@guest.booking.com,491792142916,491792142916,RETAIL,empty,Yes,OTA,Arrival Completed,July 2024
135,Karma Lake of Menteith,07 Nov 2024,KLM003081,Mr,Ignace,Verstraeten,iverst.485773@guest.booking.com,32478447631,32478447631,RETAIL,empty,Yes,OTA,Arrival Completed,empty
147,Karma Lake of Menteith,30 Jun 2024,KLM005826,empty,Iola,Skinner,iskinn.311024@guest.booking.com,447484306354,447484306354,RETAIL,empty,Yes,OTA,Arrival Completed,June 2024
149,Karma Lake of Menteith,26 Jun 2024,KLM005792,empty,Anatolii,Zotin,azotin.390589@guest.booking.com,971569132107,971569132107,RETAIL,empty,Yes,OTA,Arrival Completed,June 2024
157,Karma Lake of Menteith,19 Jun 2024,KLM005571,Mr,Graham,Tindall,gtinda.378106@guest.booking.com,447931700547,447931700547,RETAIL,empty,Yes,OTA,Arrival Completed,June 2024
199,Karma Lake of Menteith,23 May 2024,KLM005369,Ms,Oriana,Morrison,omorri.454398@guest.booking.com,447962984260,447962984260,RETAIL,empty,Yes,DIR,Arrival Completed,May 2024
204,Karma Lake of Menteith,20 May 2024,KLM003281,-,Cecile,Prince,cprinc.246474@guest.booking.com,33608482947,33608482947,RETAIL,empty,Yes,OTA,Arrival Completed,May 2024


In [ ]:
# delete email OTA
count_after_clean_OTA = m.delete_OTA(zoho_df, 'Email')
# zoho_df.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\after_clean_OTA.xlsx', index=False)

zoho_df

,Resort Name,Departure_Date,Booking Id,Booker Title,Booker First Name,Booker Last Name,Email,TelNo,Concat_Tel_Mob,Media_Group,Source,MarketingOptIn,Market_Segment,Arrival Status,Departure Date
0,Karma Lake of Menteith,31 Dec 2024,KLM007575,Mrs,Hazel,Anderson,hazel.anderson@rocketmail.com,07870229780,7870229780,RETAIL,empty,Yes,DIGITAL,Arrival Completed,December 2024
1,Karma Lake of Menteith,30 Dec 2024,KLM007452,Mr,David,Collin,dcollin07837@gmail.com,07837855439,7837855439,DIGITAL,empty,Yes,DIGITAL,Arrival Completed,December 2024
2,Karma Lake of Menteith,30 Dec 2024,KLM006985,Mr,Paul,McKenna,carole.ann95@gmail.com,empty,7990750306,RETAIL,empty,Yes,DIR,Arrival Completed,December 2024
3,Karma Lake of Menteith,26 Dec 2024,KLM007507,Mr,Alan,Riddell,alanriddell11@gmail.com,07901544137,7901544137,RETAIL,empty,Yes,DIGITAL,Arrival Completed,December 2024
4,Karma Lake of Menteith,26 Dec 2024,KLM007062,Mrs,Kathryn,Foley,dkfoley85@gmail.com,447910073758,447910073758,RETAIL,empty,Yes,DIR,Arrival Completed,December 2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506,Karma Lake of Menteith,06 Jan 2025,empty,empty,Alicia,Gwen DSouza,aliciagwendsouza@gmail.com,empty,empty,empty,empty,Yes,WEDDING,Arrival Completed,empty
507,Karma Lake of Menteith,06 Jan 2025,empty,empty,Anna,Katharina Eichhorn,anna.eichhorn@freenet.de,empty,empty,empty,empty,Yes,WEDDING,Arrival Completed,empty
508,Karma Lake of Menteith,06 Jan 2025,empty,empty,Tobias,Züchner,tobi.zuechner@icloud.com,empty,empty,empty,empty,Yes,WEDDING,Arrival Completed,empty
509,Karma Lake of Menteith,06 Jan 2025,empty,empty,Daisy,Mashburn,dafay13@gmail.com,empty,empty,empty,empty,Yes,WEDDING,Arrival Completed,empty


In [ ]:
# Cek Email Karma
email_karma = m.find_karma(zoho_df, 'Email')
count_email_karma = email_karma.shape[0]
# email_OTA.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\email_OTA.xlsx', index=False)

email_karma

,Resort Name,Departure_Date,Booking Id,Booker Title,Booker First Name,Booker Last Name,Email,TelNo,Concat_Tel_Mob,Media_Group,Source,MarketingOptIn,Market_Segment,Arrival Status,Departure Date
101,Karma Lake of Menteith,17 Aug 2024,KLM006131,Mr,Vic,Faulkner,vic.faulkner@karmagroup.com,919630852741,628113974868,RETAIL,empty,Yes,COMP,Arrival Completed,August 2024
463,Karma Lake of Menteith,06 Jan 2025,empty,empty,Susann,Bernstein,reservation@karmabavaria.com,empty,empty,empty,empty,Yes,WEDDING,Arrival Completed,empty
473,Karma Lake of Menteith,06 Jan 2025,empty,empty,Shaun,empty,gm@karmakandara.com,empty,empty,empty,empty,Yes,WEDDING,Arrival Completed,empty
475,Karma Lake of Menteith,06 Jan 2025,empty,empty,Sabine,empty,sabine.soetanto@karmagroup.com,empty,empty,empty,empty,Yes,WEDDING,Arrival Completed,empty


In [ ]:
# delete email Karma
count_after_clean_karma = m.delete_karma(zoho_df, 'Email')
# zoho_df.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\after_clean_OTA.xlsx', index=False)

zoho_df

,Resort Name,Departure_Date,Booking Id,Booker Title,Booker First Name,Booker Last Name,Email,TelNo,Concat_Tel_Mob,Media_Group,Source,MarketingOptIn,Market_Segment,Arrival Status,Departure Date
0,Karma Lake of Menteith,31 Dec 2024,KLM007575,Mrs,Hazel,Anderson,hazel.anderson@rocketmail.com,07870229780,7870229780,RETAIL,empty,Yes,DIGITAL,Arrival Completed,December 2024
1,Karma Lake of Menteith,30 Dec 2024,KLM007452,Mr,David,Collin,dcollin07837@gmail.com,07837855439,7837855439,DIGITAL,empty,Yes,DIGITAL,Arrival Completed,December 2024
2,Karma Lake of Menteith,30 Dec 2024,KLM006985,Mr,Paul,McKenna,carole.ann95@gmail.com,empty,7990750306,RETAIL,empty,Yes,DIR,Arrival Completed,December 2024
3,Karma Lake of Menteith,26 Dec 2024,KLM007507,Mr,Alan,Riddell,alanriddell11@gmail.com,07901544137,7901544137,RETAIL,empty,Yes,DIGITAL,Arrival Completed,December 2024
4,Karma Lake of Menteith,26 Dec 2024,KLM007062,Mrs,Kathryn,Foley,dkfoley85@gmail.com,447910073758,447910073758,RETAIL,empty,Yes,DIR,Arrival Completed,December 2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502,Karma Lake of Menteith,06 Jan 2025,empty,empty,Alicia,Gwen DSouza,aliciagwendsouza@gmail.com,empty,empty,empty,empty,Yes,WEDDING,Arrival Completed,empty
503,Karma Lake of Menteith,06 Jan 2025,empty,empty,Anna,Katharina Eichhorn,anna.eichhorn@freenet.de,empty,empty,empty,empty,Yes,WEDDING,Arrival Completed,empty
504,Karma Lake of Menteith,06 Jan 2025,empty,empty,Tobias,Züchner,tobi.zuechner@icloud.com,empty,empty,empty,empty,Yes,WEDDING,Arrival Completed,empty
505,Karma Lake of Menteith,06 Jan 2025,empty,empty,Daisy,Mashburn,dafay13@gmail.com,empty,empty,empty,empty,Yes,WEDDING,Arrival Completed,empty


In [ ]:
find_member_booking = zoho_df.copy()

find_member_booking.loc[find_member_booking['TelNo'].isin(df_booking['Phone1']), 'Same Value Find'] = 'Booking-Phone1'
find_member_booking.loc[find_member_booking['TelNo'].isin(df_booking['Mobile']), 'Same Value Find'] = 'Booking-Mobile'
find_member_booking.loc[find_member_booking['Concat_Tel_Mob'].isin(df_booking['Phone1']), 'Same Value Find'] = 'Booking-Phone1' 
find_member_booking.loc[find_member_booking['Concat_Tel_Mob'].isin(df_booking['Mobile']), 'Same Value Find'] = 'Booking-Mobile'
find_member_booking.loc[find_member_booking['Email'].isin(df_booking['Email']), 'Same Value Find'] = 'Booking-Email'

find_member_booking.loc[find_member_booking['TelNo'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile' 
find_member_booking.loc[find_member_booking['Concat_Tel_Mob'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile'

find_member_booking.loc[find_member_booking['Email'].isin(df_email['Email']), 'Same Value Find'] = 'Member-Email'

find_member_booking.drop(find_member_booking.loc[find_member_booking['Same Value Find'].isna()].index, inplace=True)

count_booking_dedup = find_member_booking.loc[find_member_booking['Same Value Find'].str.contains('Booking', flags=re.I, regex=True)].shape[0]
count_member_dedup = find_member_booking.loc[find_member_booking['Same Value Find'].str.contains('Member', flags=re.I, regex=True)].shape[0]

find_member_booking.replace('empty', '', inplace=True)
# find_member_booking.to_csv(r'C:\Users\fajar.fatoni\Documents\Python\Data\booking-member-dedup.csv', index=False)
# find_member_booking.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\booking-member-dedup.xlsx', index=False)

#find_member_booking.loc[find_member_booking['Same Value Find'] == 'Booking-Phone1'] 

In [ ]:
# data phone, mobile dam email member yang ada di master data 
count_wash_member = zoho_df.loc[
    zoho_df['TelNo'].isin(df_booking['Phone1']) | 
    zoho_df['TelNo'].isin(df_booking['Mobile']) |
    zoho_df['Concat_Tel_Mob'].isin(df_booking['Phone1']) | 
    zoho_df['Concat_Tel_Mob'].isin(df_booking['Mobile']) |
    zoho_df['Email'].isin(df_booking['Email'])| 

    zoho_df['TelNo'].isin(df_phone['Mobile']) | 
    zoho_df['Concat_Tel_Mob'].isin(df_phone['Mobile']) |

    zoho_df['Email'].isin(df_email['Email'])

].shape[0]

zoho_df.drop(zoho_df.loc[
    zoho_df['TelNo'].isin(df_booking['Phone1']) | 
    zoho_df['TelNo'].isin(df_booking['Mobile']) |
    zoho_df['Concat_Tel_Mob'].isin(df_booking['Phone1']) | 
    zoho_df['Concat_Tel_Mob'].isin(df_booking['Mobile']) |
    zoho_df['Email'].isin(df_booking['Email']) |
    
    zoho_df['TelNo'].isin(df_phone['Mobile']) | 
    zoho_df['Concat_Tel_Mob'].isin(df_phone['Mobile']) |

    zoho_df['Email'].isin(df_email['Email'])

    ].index, inplace=True)
    
zoho_df.reset_index(drop=True, inplace=True)

count_after_wash_member = zoho_df.shape[0]


In [ ]:
count_email_only = zoho_df.loc[(zoho_df['Email'] != 'empty') & ((zoho_df['TelNo'] == 'empty') & (zoho_df['Concat_Tel_Mob'] == 'empty'))].shape[0]
count_phone_only = zoho_df.loc[(zoho_df['Email'] == 'empty') & ((zoho_df['TelNo'] != 'empty') | (zoho_df['Concat_Tel_Mob'] != 'empty'))].shape[0]
count_email_and_phone = zoho_df.loc[(zoho_df['Email'] != 'empty') & ((zoho_df['TelNo'] != 'empty') | (zoho_df['Concat_Tel_Mob'] != 'empty'))].shape[0]

count_blank = zoho_df.loc[(zoho_df['Email'] =='empty') & (zoho_df['TelNo']=='empty') & (zoho_df['Concat_Tel_Mob']=='empty')].shape[0]

# zoho_df.loc[(zoho_df['Email'] =='empty') & (zoho_df['TelNo']=='empty') & (zoho_df['Concat_Tel_Mob']=='empty')].to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\blank_data.xlsx', index=False)

zoho_df.loc[(zoho_df['Email'] != 'empty') & ((zoho_df['TelNo'] == 'empty') & (zoho_df['Concat_Tel_Mob'] == 'empty')), 'Contact Type'] = 'Email Only'
zoho_df.loc[(zoho_df['Email'] == 'empty') & ((zoho_df['TelNo'] != 'empty') | (zoho_df['Concat_Tel_Mob'] != 'empty')), 'Contact Type'] = 'Phone Only'
zoho_df.loc[(zoho_df['Email'] != 'empty') & ((zoho_df['TelNo'] != 'empty') | (zoho_df['Concat_Tel_Mob'] != 'empty')), 'Contact Type'] = 'Email and Phone'

zoho_df.drop(zoho_df.loc[(zoho_df['Email'] =='empty') & (zoho_df['TelNo']=='empty') & (zoho_df['Concat_Tel_Mob']=='empty')].index, inplace=True)
zoho_df.reset_index(drop=True, inplace=True)

count_after_wash_blank = zoho_df.shape[0]

In [ ]:
arr_dedup = [count_deleted_wrong_format, count_duplicate_general, count_leads_test, count_email_OTA, count_email_karma, count_blank]
count_sum_dedup = sum(arr_dedup)
count_sum_dedup

135

In [ ]:
print("==========================================================")
print("Summary")
print("==========================================================")
print("Jumlah Filter Leads Awal : "+str(count_filter_awal))
print("Jumlah Format Email Salah : "+str(count_deleted_wrong_format))
print("Jumlah Setelah Clean Format Email Salah : "+str(count_after_deleted_wrong_format))
print("Jumlah Duplicate (Phone, Mobile dan Email) : "+str(count_duplicate_general))
print("Jumlah Setelah Clean Duplicate : "+str(count_after_dedup_duplicate))
print("Jumlah Lead Test : "+str(count_leads_test))
print("Jumlah Setelah Clean Lead Test : "+str(count_after_dedup_test))
# print("Jumlah Lead Do Not Mail : "+str(count_dnm))
# print("Jumlah Setelah Clean Lead Do Not Mail : "+str(count_after_dedup_dnm))
print("Jumlah Email OTA : "+str(count_email_OTA))
print("Jumlah Setelah Clean Email OTA : "+str(count_after_clean_OTA))
print("Jumlah Email Karma : "+str(count_email_karma))
print("Jumlah Setelah Clean Email Karma : "+str(count_after_clean_karma))
print("Jumlah Wash Member : "+str(count_member_dedup))
print("Jumlah Wash Booking : "+str(count_booking_dedup))
print("Jumlah Setelah Wash Member : "+str(count_after_wash_member))
print("Jumlah Phone Email Blank (Setelah phone yang tidak valid dihapus) : "+str(count_blank))
print("Jumlah Setelah Wash Phone Email Blank : "+str(count_after_wash_blank))
print("==========================================================")
print("Contact Type")
print("==========================================================")
print("Jumlah Email Only : "+str(count_email_only))
print("Jumlah Phone Only : "+str(count_phone_only))
print("Jumlah Email dan Phone : "+str(count_email_and_phone))

Summary
Jumlah Filter Leads Awal : 641
Jumlah Format Email Salah : 1
Jumlah Setelah Clean Format Email Salah : 640
Jumlah Duplicate (Phone, Mobile dan Email) : 92
Jumlah Setelah Clean Duplicate : 548
Jumlah Lead Test : 2
Jumlah Setelah Clean Lead Test : 546
Jumlah Email OTA : 35
Jumlah Setelah Clean Email OTA : 511
Jumlah Email Karma : 4
Jumlah Setelah Clean Email Karma : 507
Jumlah Wash Member : 18
Jumlah Wash Booking : 0
Jumlah Setelah Wash Member : 489
Jumlah Phone Email Blank (Setelah phone yang tidak valid dihapus) : 1
Jumlah Setelah Wash Phone Email Blank : 488
Contact Type
Jumlah Email Only : 144
Jumlah Phone Only : 10
Jumlah Email dan Phone : 334


In [ ]:
print("==========================================================")
print("Summary Report")
print("==========================================================")
print("Jumlah Data Awal : "+str(count_filter_awal))
print("Jumlah Wash Dedup : "+str(count_sum_dedup))
print("Jumlah Wash Member : "+str(count_member_dedup))
print("Jumlah Wash Booking : "+str(count_booking_dedup))
print("Jumlah Data Akhir : "+str(count_after_wash_blank))

Summary Report
Jumlah Data Awal : 641
Jumlah Wash Dedup : 135
Jumlah Wash Member : 18
Jumlah Wash Booking : 0
Jumlah Data Akhir : 488


In [ ]:
zoho_df.replace('empty', '', inplace=True, regex=True)

zoho_df['Booker First Name'] = zoho_df['Booker First Name'].str.title()
zoho_df['Booker Last Name'] = zoho_df['Booker Last Name'].str.title()

zoho_df.to_csv(r'C:\Users\fajar.fatoni\Documents\Python\Data\KLOM.csv', index=False)
# zoho_df.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\newsletter-clean.xlsx', index=False)